<a href="https://colab.research.google.com/github/strakovskay/yolo/blob/main/yolov11schoolbus1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# YOLOv11 School Bus Detection - Google Colab
# ==========================================

# ----- Cell 1: التثبيت -----
!pip uninstall numpy -y -q
!pip install "numpy<2.0" ultralytics==8.3.40 -q

print("✅ تم التثبيت!")
print("⚠️ أعد تشغيل Runtime: Runtime → Restart Runtime")
print("ثم شغل Cell 2 وما بعده")

# ----- Cell 2: رفع البيانات -----
from google.colab import files
import zipfile
import os

print("📤 ارفع ملف ZIP الخاص بالبيانات")
uploaded = files.upload()

# فك الضغط
for filename in uploaded.keys():
    print(f"\n📦 فك ضغط {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

print("\n✅ تم رفع البيانات!")
print("📁 الملفات في: /content/dataset")
!ls -la /content/dataset

# ----- Cell 3: الاستيراد والإعداد -----
import os
import shutil
import yaml
import torch
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np

print(f"✅ الاستيراد نجح!")
print(f"🖥️ GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   {torch.cuda.get_device_name(0)}")

# تحديد مسار البيانات
# عدّل هذا المسار حسب موقع ملف data.yaml
DATA_PATH = "/content/dataset"  # أو المسار الصحيح بعد فك الضغط

print(f"\n📂 محتويات المجلد:")
!ls -la {DATA_PATH}

# ----- Cell 4: نسخ البيانات النظيفة -----
def copy_dataset(input_path, dest_dir="/content/clean_dataset"):
    print("\n🔄 نسخ البيانات...")

    input_base = Path(input_path)
    dest_base = Path(dest_dir)
    dest_base.mkdir(exist_ok=True, parents=True)

    # البحث عن data.yaml
    yaml_files = list(input_base.rglob('data.yaml'))
    if yaml_files:
        yaml_path = yaml_files[0]
        print(f"✓ وجد data.yaml في: {yaml_path}")
    else:
        print("❌ لم يتم العثور على data.yaml")
        return None

    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)

    print(f"\n--- محتوى data.yaml ---")
    print(yaml.dump(config, default_flow_style=False))
    print("------------------------")

    stats = {'total': 0, 'copied': 0}
    new_config = {
        'nc': config['nc'],
        'names': config['names'],
        'train': 'train/images',
        'val': 'val/images'
    }

    splits = {
        'train': ['train', 'training'],
        'val': ['valid', 'val', 'validation'],
        'test': ['test']
    }

    yaml_parent = yaml_path.parent

    for split, names in splits.items():
        print(f"\n  📁 {split}:")

        src_img = None
        src_lbl = None

        for name in names:
            test = yaml_parent / name / 'images'
            if test.exists():
                src_img = test
                src_lbl = yaml_parent / name / 'labels'
                break

        if not src_img:
            if split == 'test':
                continue
            print(f"     ⚠️ لم يوجد")
            continue

        print(f"     ✓ {src_img}")

        dst_img = dest_base / split / 'images'
        dst_lbl = dest_base / split / 'labels'
        dst_img.mkdir(exist_ok=True, parents=True)
        dst_lbl.mkdir(exist_ok=True, parents=True)

        images = list(src_img.glob('*.jpg')) + list(src_img.glob('*.png')) + list(src_img.glob('*.jpeg'))

        copied = 0
        for img in images:
            stats['total'] += 1
            try:
                Image.open(img).verify()
                shutil.copy2(img, dst_img / img.name)

                lbl = src_lbl / (img.stem + '.txt')
                if lbl.exists():
                    shutil.copy2(lbl, dst_lbl / lbl.name)

                stats['copied'] += 1
                copied += 1
            except:
                pass

        print(f"     ✅ {copied} صورة")

        if split in ['train', 'val']:
            new_config[split] = f'{split}/images'
        elif split == 'test' and copied > 0:
            new_config['test'] = f'{split}/images'

    new_yaml = dest_base / 'data.yaml'
    with open(new_yaml, 'w') as f:
        yaml.dump(new_config, f, default_flow_style=False)

    print(f"\n📊 المجموع: {stats['total']} | ✅ {stats['copied']}")
    print(f"\n✅ {new_yaml}")

    if stats['copied'] == 0:
        raise Exception("❌ لا توجد صور!")

    return str(new_yaml)

CLEAN_YAML = copy_dataset(DATA_PATH)
print("\n✅ جاهز للتدريب!")

# ----- Cell 5: التدريب -----
print("\n" + "="*50)
print("🚀 بدء التدريب")
print("="*50 + "\n")

# إصلاح Ray Tune
import ultralytics.utils.callbacks.raytune as raytune_file
raytune_file.on_fit_epoch_end = lambda x: None
raytune_file.on_train_end = lambda x: None

model = YOLO('yolo11n.pt')

results = model.train(
    data=CLEAN_YAML,
    epochs=50,
    imgsz=640,
    batch=16,

    amp=True,  # Colab يدعم AMP
    workers=2,  # Colab يمكنه استخدام workers
    cache=False,

    augment=True,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,

    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,

    patience=20,
    save=True,
    save_period=10,

    project='/content/runs/detect',
    name='school_bus',
    exist_ok=True,
    plots=True,
    verbose=True,

    device=0 if torch.cuda.is_available() else 'cpu'
)

print(f"\n✅ انتهى!")
print(f"📁 {results.save_dir}")

# ----- Cell 6: التقييم -----
print("\n📊 التقييم:")

best = YOLO(f"{results.save_dir}/weights/best.pt")
val = best.val(data=CLEAN_YAML)

print(f"\n{'='*40}")
print(f"mAP50:     {val.box.map50:.4f}")
print(f"mAP50-95:  {val.box.map:.4f}")
print(f"Precision: {val.box.mp:.4f}")
print(f"Recall:    {val.box.mr:.4f}")
print(f"{'='*40}")

# ----- Cell 7: عرض النتائج -----
from IPython.display import Image as IPImage, display

print("\n📊 الرسوم البيانية:")

plots = ['results.png', 'confusion_matrix.png', 'PR_curve.png', 'F1_curve.png']

for plot in plots:
    path = f"{results.save_dir}/{plot}"
    if os.path.exists(path):
        print(f"\n--- {plot} ---")
        display(IPImage(filename=path, width=800))

# ----- Cell 8: اختبار النموذج -----
print("\n🧪 اختبار النموذج:")

val_imgs = list(Path(CLEAN_YAML).parent.glob('val/images/*.jpg'))
if val_imgs:
    test_img = str(val_imgs[0])
    print(f"📸 {Path(test_img).name}")

    pred = best.predict(
        source=test_img,
        conf=0.25,
        save=True,
        project='/content/runs/predict',
        name='test',
        exist_ok=True
    )

    result = f'/content/runs/predict/test/{Path(test_img).name}'
    if os.path.exists(result):
        display(IPImage(filename=result, width=800))

print("\n✅ اكتمل كل شيء!")
print(f"🏆 أفضل نموذج: {results.save_dir}/weights/best.pt")

# ----- Cell 9: تحميل النموذج -----
from google.colab import files

print("\n📥 تحميل النموذج المدرب:")

# تحميل أفضل نموذج
files.download(f"{results.save_dir}/weights/best.pt")

# تحميل النتائج
if os.path.exists(f"{results.save_dir}/results.csv"):
    files.download(f"{results.save_dir}/results.csv")

print("✅ تم التحميل!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
✅ تم التثبيت!
⚠️ أعد تشغيل Runtime: Runtime → Restart Runtime
ثم شغل Cell 2 وما بعده
📤 ارفع ملف ZIP الخاص بالبيانات


KeyboardInterrupt: 

In [ ]:
# ----- Cell 1: إعادة تشغيل تلقائية -----
!pip uninstall opencv-python opencv-python-headless opencv-contrib-python numpy -y -q
!pip install "numpy==1.26.4" opencv-python ultralytics==8.3.40 -q

print("✅ تم التثبيت!")
print("🔄 إعادة تشغيل Runtime تلقائياً...")

# إعادة تشغيل تلقائية
import os
os.kill(os.getpid(), 9)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# ----- Cell 1 البديلة (أسهل) -----
!pip install --upgrade ultralytics -q

print("✅ تم التثبيت!")
print("⚠️ أعد تشغيل Runtime: Runtime → Restart Runtime")
print("ثم شغل Cell 2 وما بعده")

✅ تم التثبيت!
⚠️ أعد تشغيل Runtime: Runtime → Restart Runtime
ثم شغل Cell 2 وما بعده


In [ ]:
# ----- الكود الكامل بعد إعادة التشغيل -----
from google.colab import files
import zipfile
import os
import shutil
import yaml
import torch
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np

print(f"✅ NumPy: {np.__version__}")
print(f"🖥️ GPU: {torch.cuda.is_available()}")

# رفع البيانات
print("\n📤 ارفع ملف ZIP")
uploaded = files.upload()

for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

print("✅ تم رفع البيانات!")

# نسخ البيانات
def copy_dataset(input_path="/content/dataset", dest_dir="/content/clean"):
    Path(dest_dir).mkdir(exist_ok=True)

    yaml_path = Path(input_path) / "data.yaml"
    with open(yaml_path) as f:
        config = yaml.safe_load(f)

    new_config = {'nc': config['nc'], 'names': config['names']}
    stats = {'total': 0, 'copied': 0}

    for split, folder in [('train', 'train'), ('val', 'valid'), ('test', 'test')]:
        src_img = Path(input_path) / folder / 'images'
        src_lbl = Path(input_path) / folder / 'labels'

        if not src_img.exists():
            continue

        dst_img = Path(dest_dir) / split / 'images'
        dst_lbl = Path(dest_dir) / split / 'labels'
        dst_img.mkdir(parents=True, exist_ok=True)
        dst_lbl.mkdir(parents=True, exist_ok=True)

        for img in src_img.glob('*.jpg'):
            stats['total'] += 1
            try:
                shutil.copy2(img, dst_img / img.name)
                lbl = src_lbl / (img.stem + '.txt')
                if lbl.exists():
                    shutil.copy2(lbl, dst_lbl / lbl.name)
                stats['copied'] += 1
            except:
                pass

        if split in ['train', 'val']:
            new_config[split] = f'{split}/images'

    new_yaml = Path(dest_dir) / 'data.yaml'
    with open(new_yaml, 'w') as f:
        yaml.dump(new_config, f)

    print(f"📊 {stats['copied']}/{stats['total']} صورة")
    return str(new_yaml)

DATA_YAML = copy_dataset()

# التدريب
print("\n🚀 بدء التدريب...")

import ultralytics.utils.callbacks.raytune as rt
rt.on_fit_epoch_end = lambda x: None

model = YOLO('yolo11n.pt')

results = model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    amp=True,
    workers=2,
    patience=20,
    project='/content/runs',
    name='school_bus',
    exist_ok=True,
    device=0
)

print(f"✅ انتهى! {results.save_dir}")

# النتائج
best = YOLO(f"{results.save_dir}/weights/best.pt")
val = best.val(data=DATA_YAML)
print(f"\nmAP50: {val.box.map50:.4f}")
print(f"mAP50-95: {val.box.map:.4f}")

# تحميل النموذج
files.download(f"{results.save_dir}/weights/best.pt")

✅ NumPy: 1.26.4
🖥️ GPU: True

📤 ارفع ملف ZIP


Saving school bus.yolov11.zip to school bus.yolov11.zip
✅ تم رفع البيانات!
📊 1391/1391 صورة

🚀 بدء التدريب...
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/clean/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=school_bus, nbs=64, nms=False, opset

In [ ]:
from ultralytics import YOLO
from IPython.display import Image, display

# تحميل النموذج
model = YOLO('/content/runs/school_bus/weights/best.pt')

# اختبار على صورة
results = model.predict(
    source='/content/clean/val/images',  # مجلد الصور
    conf=0.25,
    save=True
)

print("✅ النتائج في: runs/predict/predict")

In [ ]:
from ultralytics import YOLO
from IPython.display import Image as IPImage, display
import os
from pathlib import Path

# تحميل النموذج
print("🔄 تحميل النموذج...")
model = YOLO('/content/runs/school_bus/weights/best.pt')

# اختبار على مجلد الصور
print("🧪 اختبار النموذج...")
results = model.predict(
    source='/content/clean/val/images',
    conf=0.25,
    save=True,
    project='runs/predict',
    name='val_results',
    exist_ok=True
)

print(f"✅ النتائج في: runs/predict/val_results")

# عرض أول 5 صور من النتائج
print("\n📸 عرض بعض النتائج:")
result_dir = Path('runs/predict/val_results')
result_images = list(result_dir.glob('*.jpg'))[:5]

for i, img_path in enumerate(result_images, 1):
    print(f"\n--- الصورة {i}/{len(result_images)} ---")
    display(IPImage(filename=str(img_path), width=700))

print(f"\n✅ تم اكتشاف {len(result_images)} صورة!")
print(f"📁 جميع النتائج في: {result_dir}")

# إحصائيات الاكتشافات
print("\n📊 إحصائيات:")
total_detections = sum(len(r.boxes) for r in results)
print(f"إجمالي الاكتشافات: {total_detections}")
print(f"عدد الصور: {len(results)}")
print(f"متوسط الاكتشافات لكل صورة: {total_detections/len(results):.2f}")

In [ ]:
# ========================================
# نموذج كشف الحافلات المدرسية - YOLOv11l
# ========================================

import yaml
import os
from pathlib import Path

print("🔄 إعداد البيانات للحافلات المدرسية فقط...")

# ----- 1. إنشاء data.yaml الجديد -----
new_config = {
    'nc': 1,
    'names': ['School-Bus'],
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images'
}

with open('/content/clean/data_schoolbus_only.yaml', 'w') as f:
    yaml.dump(new_config, f, default_flow_style=False)

print("✅ تم إنشاء: data_schoolbus_only.yaml")

# ----- 2. تحديث ملفات التسميات -----
def convert_to_schoolbus_only(labels_dir):
    """
    تحويل جميع التسميات:
    - School-bus (1) و SchoolBus (2) → 0 (School-Bus)
    - حذف Bus (0), car (3), police (4)
    """
    count = {'kept': 0, 'removed': 0, 'files': 0}

    for label_file in Path(labels_dir).glob('*.txt'):
        with open(label_file, 'r') as f:
            lines = f.readlines()

        new_lines = []

        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])

                # الاحتفاظ فقط بـ School-bus (1) و SchoolBus (2)
                if class_id == 1 or class_id == 2:
                    parts[0] = '0'  # تحويل إلى class 0
                    new_lines.append(' '.join(parts) + '\n')
                    count['kept'] += 1
                else:
                    count['removed'] += 1

        # حفظ الملف (حتى لو فارغ)
        with open(label_file, 'w') as f:
            f.writelines(new_lines)

        count['files'] += 1

    return count

# تطبيق على جميع المجلدات
print("\n🔄 تحديث ملفات التسميات...")
total_kept = 0
total_removed = 0

for split in ['train', 'val', 'test']:
    labels_dir = f'/content/clean/{split}/labels'
    if os.path.exists(labels_dir):
        stats = convert_to_schoolbus_only(labels_dir)
        print(f"\n📁 {split}:")
        print(f"   ✅ محتفظ بـ: {stats['kept']} حافلة مدرسية")
        print(f"   ❌ محذوف: {stats['removed']} عنصر آخر")
        print(f"   📄 ملفات: {stats['files']}")
        total_kept += stats['kept']
        total_removed += stats['removed']

print(f"\n{'='*50}")
print(f"📊 المجموع:")
print(f"   ✅ حافلات مدرسية: {total_kept}")
print(f"   ❌ محذوفات: {total_removed}")
print(f"{'='*50}")

# ----- 3. بدء التدريب مع YOLOv11l -----
print("\n🚀 تحميل YOLOv11l...")
print("📦 حجم النموذج: ~50MB (أكبر من n لكن أدق)")

from ultralytics import YOLO
import ultralytics.utils.callbacks.raytune as rt
rt.on_fit_epoch_end = lambda x: None

# استخدام YOLOv11l بدلاً من n
model = YOLO('yolo11l.pt')

print("\n🔥 بدء التدريب مع YOLOv11l...")
print("⏱️ سيستغرق وقت أطول لكن النتائج أفضل!")

results = model.train(
    data='/content/clean/data_schoolbus_only.yaml',
    epochs=50,
    imgsz=640,
    batch=8,  # أصغر من n بسبب حجم النموذج

    # إعدادات محسّنة
    amp=True,
    workers=2,
    cache=False,

    # Data Augmentation
    augment=True,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.5,

    # Optimizer
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,

    # Early stopping
    patience=20,
    save=True,
    save_period=10,

    # Output
    project='/content/runs/detect',
    name='schoolbus_yolo11l',
    exist_ok=True,
    plots=True,
    verbose=True,

    device=0
)

print(f"\n{'='*50}")
print("✅ التدريب اكتمل!")
print(f"📁 النتائج: {results.save_dir}")
print(f"{'='*50}")

# ----- 4. التقييم -----
print("\n📊 تقييم النموذج...")

best = YOLO(f"{results.save_dir}/weights/best.pt")
val = best.val(data='/content/clean/data_schoolbus_only.yaml')

print(f"\n{'='*40}")
print("🎯 المقاييس النهائية:")
print(f"{'='*40}")
print(f"mAP50:     {val.box.map50:.4f} ({val.box.map50*100:.1f}%)")
print(f"mAP50-95:  {val.box.map:.4f} ({val.box.map*100:.1f}%)")
print(f"Precision: {val.box.mp:.4f} ({val.box.mp*100:.1f}%)")
print(f"Recall:    {val.box.mr:.4f} ({val.box.mr*100:.1f}%)")
print(f"{'='*40}")

# ----- 5. اختبار -----
print("\n🧪 اختبار النموذج...")

test_results = best.predict(
    source='/content/clean/val/images',
    conf=0.25,
    save=True,
    project='runs/predict',
    name='schoolbus_l_test',
    exist_ok=True
)

print(f"✅ النتائج في: runs/predict/schoolbus_l_test")

# عرض بعض النتائج
from IPython.display import Image as IPImage, display

print("\n📸 عرض النتائج:")
result_dir = Path('runs/predict/schoolbus_l_test')
result_images = list(result_dir.glob('*.jpg'))[:3]

for i, img_path in enumerate(result_images, 1):
    print(f"\n--- الصورة {i} ---")
    display(IPImage(filename=str(img_path), width=700))

# إحصائيات
total_detections = sum(len(r.boxes) for r in test_results)
print(f"\n📊 تم كشف {total_detections} حافلة مدرسية في {len(test_results)} صورة")

# ----- 6. تحميل النموذج -----
from google.colab import files

print("\n📥 تحميل النموذج...")
files.download(f"{results.save_dir}/weights/best.pt")

# تحميل النتائج أيضاً
if os.path.exists(f"{results.save_dir}/results.csv"):
    files.download(f"{results.save_dir}/results.csv")

print("\n✅ اكتمل كل شيء!")

# معلومات النموذج
print(f"\n{'='*50}")
print("📊 مقارنة YOLOv11l vs YOLOv11n:")
print(f"{'='*50}")
print("YOLOv11n: 2.6M معاملات | سريع | دقة جيدة")
print("YOLOv11l: 25M معاملات | أبطأ | دقة ممتازة ✅")
print(f"{'='*50}")

print(f"\n💡 لاستخدام النموذج لاحقاً:")
print(f"model = YOLO('best.pt')")
print(f"results = model.predict('image.jpg', conf=0.25)")

In [ ]:
# ==========================================
# تحسين نموذج كشف الحافلات المدرسية
# ==========================================

from ultralytics import YOLO
import ultralytics.utils.callbacks.raytune as rt
rt.on_fit_epoch_end = lambda x: None

print("🔥 تدريب محسّن - YOLOv11l")

# تحميل النموذج المُدرّب سابقاً
model = YOLO('/content/runs/detect/schoolbus_yolo11l/weights/best.pt')

# التدريب المحسّن
results = model.train(
    data='/content/clean/data_schoolbus_only.yaml',

    # ===== 1. زيادة Epochs =====
    epochs=100,  # بدلاً من 50
    patience=30,  # بدلاً من 20

    # ===== 2. تحسين Batch و Image Size =====
    imgsz=640,
    batch=8,  # أكبر من 8 (إذا سمحت الذاكرة)

    # ===== 3. Data Augmentation أقوى =====
    augment=True,
    mosaic=1.0,
    mixup=0.15,      # إضافة mixup
    copy_paste=0.1,  # إضافة copy-paste
    degrees=15.0,    # زيادة الدوران
    translate=0.2,   # زيادة الإزاحة
    scale=0.9,       # تكبير/تصغير أكثر
    shear=5.0,       # إضافة shear
    perspective=0.0001,  # إضافة perspective
    flipud=0.1,      # انعكاس عمودي
    fliplr=0.5,
    hsv_h=0.02,      # تغيير ألوان أكثر
    hsv_s=0.8,
    hsv_v=0.5,
    erasing=0.5,     # random erasing

    # ===== 4. Optimizer محسّن =====
    optimizer='AdamW',
    lr0=0.0005,      # learning rate أصغر (fine-tuning)
    lrf=0.001,
    momentum=0.95,
    weight_decay=0.001,
    warmup_epochs=5.0,

    # ===== 5. إعدادات إضافية =====
    amp=True,
    workers=4,  # أكثر workers
    cache='ram',  # تخزين في RAM للسرعة

    close_mosaic=20,  # إغلاق mosaic في آخر 20 epoch

    # Loss weights
    box=7.5,
    cls=0.5,
    dfl=1.5,

    # Multi-scale training
    multi_scale=True,

    # Output
    project='/content/runs/detect',
    name='schoolbus_yolo11l_v2',
    exist_ok=True,
    plots=True,
    save=True,
    save_period=10,
    verbose=True,

    device=0,
    resume=False  # تدريب جديد من best.pt
)

print("\n✅ التدريب المحسّن اكتمل!")
print(f"📁 {results.save_dir}")

# التقييم
best = YOLO(f"{results.save_dir}/weights/best.pt")
val = best.val(data='/content/clean/data_schoolbus_only.yaml')

print(f"\n{'='*50}")
print("🎯 النتائج المحسّنة:")
print(f"{'='*50}")
print(f"mAP50:     {val.box.map50:.4f} ({val.box.map50*100:.1f}%)")
print(f"mAP50-95:  {val.box.map:.4f} ({val.box.map*100:.1f}%)")
print(f"Precision: {val.box.mp:.4f} ({val.box.mp*100:.1f}%)")
print(f"Recall:    {val.box.mr:.4f} ({val.box.mr*100:.1f}%)")
print(f"{'='*50}")

# مقارنة
print(f"\n📈 التحسين:")
print(f"mAP50:     0.815 → {val.box.map50:.3f} ({(val.box.map50-0.815)*100:+.1f}%)")
print(f"mAP50-95:  0.610 → {val.box.map:.3f} ({(val.box.map-0.610)*100:+.1f}%)")
print(f"Precision: 0.768 → {val.box.mp:.3f} ({(val.box.mp-0.768)*100:+.1f}%)")
print(f"Recall:    0.828 → {val.box.mr:.3f} ({(val.box.mr-0.828)*100:+.1f}%)")

# تحميل
from google.colab import files
files.download(f"{results.save_dir}/weights/best.pt")

In [ ]:
# ==========================================
# حل مشكلة Out of Memory
# ==========================================

# 1️⃣ تنظيف الذاكرة أولاً
import gc
import torch

torch.cuda.empty_cache()
gc.collect()

print("✅ تم تنظيف الذاكرة")

# 2️⃣ التدريب بإعدادات أخف
from ultralytics import YOLO
import ultralytics.utils.callbacks.raytune as rt
rt.on_fit_epoch_end = lambda x: None

print("🔥 تدريب محسّن - إعدادات متوازنة")

model = YOLO('/content/runs/detect/schoolbus_yolo11l/weights/best.pt')

results = model.train(
    data='/content/clean/data_schoolbus_only.yaml',

    # ===== تقليل استهلاك الذاكرة =====
    epochs=80,
    patience=25,
    batch=8,  # ✅ أصغر من 16
    imgsz=640,

    # ===== تعطيل cache في RAM =====
    cache=False,  # ✅ بدلاً من 'ram'
    workers=2,  # ✅ بدلاً من 4

    # ===== تقليل multi-scale =====
    multi_scale=False,  # ✅ تعطيل لتوفير الذاكرة

    # ===== Augmentation محسّن =====
    augment=True,
    mosaic=1.0,
    mixup=0.1,  # ✅ أقل من 0.15
    copy_paste=0.05,  # ✅ أقل من 0.1
    degrees=12.0,
    translate=0.15,
    scale=0.8,
    shear=3.0,
    perspective=0.0,  # ✅ تعطيل
    flipud=0.0,  # ✅ تعطيل
    fliplr=0.5,
    hsv_h=0.02,
    hsv_s=0.7,
    hsv_v=0.4,
    erasing=0.3,  # ✅ أقل من 0.5

    # ===== Optimizer =====
    optimizer='AdamW',
    lr0=0.0005,
    lrf=0.001,
    momentum=0.95,
    weight_decay=0.001,
    warmup_epochs=5.0,

    # ===== AMP =====
    amp=True,

    close_mosaic=15,

    # ===== Output =====
    project='/content/runs/detect',
    name='schoolbus_yolo11l_improved',
    exist_ok=True,
    plots=True,
    save=True,
    save_period=10,
    verbose=True,

    device=0
)

print("\n✅ التدريب المحسّن اكتمل!")
print(f"📁 {results.save_dir}")

# التقييم
best = YOLO(f"{results.save_dir}/weights/best.pt")
val = best.val(data='/content/clean/data_schoolbus_only.yaml')

print(f"\n{'='*50}")
print("🎯 النتائج المحسّنة:")
print(f"{'='*50}")
print(f"mAP50:     {val.box.map50:.4f} ({val.box.map50*100:.1f}%)")
print(f"mAP50-95:  {val.box.map:.4f} ({val.box.map*100:.1f}%)")
print(f"Precision: {val.box.mp:.4f} ({val.box.mp*100:.1f}%)")
print(f"Recall:    {val.box.mr:.4f} ({val.box.mr*100:.1f}%)")
print(f"{'='*50}")

# مقارنة
print(f"\n📈 التحسين:")
print(f"mAP50:     0.815 → {val.box.map50:.3f} ({(val.box.map50-0.815)*100:+.1f}%)")
print(f"Precision: 0.768 → {val.box.mp:.3f} ({(val.box.mp-0.768)*100:+.1f}%)")
print(f"Recall:    0.828 → {val.box.mr:.3f} ({(val.box.mr-0.828)*100:+.1f}%)")

# تحميل
from google.colab import files
files.download(f"{results.save_dir}/weights/best.pt")

print("\n✅ اكتمل!")